In [ ]:
import os
import re
import time
import pandas as pd

from dataclasses import dataclass
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [ ]:
from typing import Optional, List, Tuple, Union

In [ ]:
import sys

In [ ]:
_PKG_PATH: str = "/Users/adebayobraimah/Desktop/projects/CSE505"

In [ ]:
sys.path.append(_PKG_PATH)

In [ ]:
from src.utils import util

In [ ]:
url = "https://prod.ps.stonybrook.edu/psc/csprodg/EMPLOYEE/CAMP/c/COMMUNITY_ACCESS.SSS_BROWSE_CATLG.GBL?"

In [ ]:
wait_time: int = 10

In [ ]:
# Setup Selenium WebDriver
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
major_three_letter_code: str = "mat".upper()
major_three_letter_code

In [ ]:
nav_letter: str = major_three_letter_code[0].upper()
nav_letter

In [ ]:
WebDriverWait(driver, wait_time).until(
    EC.element_to_be_clickable((By.LINK_TEXT, nav_letter))
).click()

In [ ]:
# Navigate to major
WebDriverWait(driver, wait_time).until(
    EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, major_three_letter_code))
).click()

In [ ]:
# Get table data
time.sleep(wait_time // 2)  # Time to wait for javascript to load the table.
table: List[webdriver.remote.webelement.WebElement] = driver.find_elements(
    By.TAG_NAME, "tbody"
)

In [ ]:
for tab in reversed(table):
    try:
        tab.find_element(By.PARTIAL_LINK_TEXT, "Course Nbr")
    except (AttributeError, NoSuchElementException):
        table.remove(tab)

try:
    table = table[-1]  # Get the last table
except IndexError:
    table = None

In [ ]:
table

In [ ]:
# Verify table
if not table.is_displayed():
    raise ValueError("Table is not displayed. Check the URL and major code.")

if not table.text:
    raise ValueError("Table is empty. Check the URL and major code.")


In [ ]:
headers = [header.text for header in table.find_elements(By.TAG_NAME, "th")]
headers

In [ ]:
rows = []
for row in table.find_elements(By.TAG_NAME, "tr"):
    cells = [cell.text for cell in row.find_elements(By.TAG_NAME, "td")]
    if cells:  # This check is to skip rows without table data cells
        rows.append(cells)
rows

In [ ]:
df = pd.DataFrame(rows, columns=headers)

In [ ]:
_course_numbers: List[str] = df["Course Nbr"].tolist()
# _course_numbers

In [ ]:
# NO EDIT
def clean_course_title(course_title: str) -> str:
    """Clean course title by removing any additional information after '**'.

    Args:
        course_title: Course title string.

    Returns:
        Cleaned course title string.
    """
    # Use a regular expression to match only the course title before '**'
    cleaned_title = re.sub(r"\*\*.*$", "", course_title).strip()
    return cleaned_title

In [ ]:
_course_titles: List[str] = df["Course Title"].tolist()
df["Course Title"] = [clean_course_title(title) for title in _course_titles]

In [ ]:
course = _course_numbers[0]
course

In [ ]:
# Wait for the page to load and click on course number
WebDriverWait(driver, wait_time).until(
    EC.element_to_be_clickable((By.LINK_TEXT, f"{course}"))
).click()

In [ ]:
# Use ID to find element -- it is unique.
WebDriverWait(driver, wait_time).until(
    EC.presence_of_element_located(
        (By.ID, "win0divSSR_CRSE_OFF_VW_ACAD_CAREER$0")
    )
).click()

In [ ]:
career: str = driver.find_element(By.ID, "win0divSSR_CRSE_OFF_VW_ACAD_CAREER$0").text
career

In [ ]:
units: str = float(driver.find_element(By.ID, "DERIVED_CRSECAT_UNITS_RANGE$0").text)
units

In [ ]:
grading_basis: str = driver.find_element(
    By.ID, "win0divSSR_CRSE_OFF_VW_GRADING_BASIS$0"
).text
grading_basis

In [ ]:
_enrollment_requirement: str = driver.find_element(
    By.ID, "DERIVED_CRSECAT_DESCR254A$0"
).text
_enrollment_requirement

In [ ]:
academic_group: str = driver.find_element(By.ID, "ACAD_GROUP_TBL_DESCR$0").text
academic_group